# Spotify Data Analysis
---

## 1. Required Libraries
---

In [2]:
import requests
import base64
import datetime
from urllib.parse import urlencode

In [3]:
from IPython.display import Image
from IPython.core.display import HTML 

## 2. Helper Functions
---

In [4]:
%load_ext autoreload
%autoreload 2
import helper_functions as hf

### 2.1 Description
---

* `auth()`:
* `search_spotify()`: The parameters that these functions accept are
* `get_artist_id()`:
* `get_albums()`:
* `get_tracks()`:
* `all_artists_songs()`:
* `all_songs()`:

## 3. Set-Up
---

### 3.1 Access Token
---
Depending on the level of access you want, there are several ways of interacting with the Spotify API.
For my personal workflow I created a function called `auth()`. This function handles all the steps that needed to be fullfilled to get the access token. When called, the function will ask for a `Client_ID` and a `Client_Secret`. You can learn how to get those access in this [**article**](https://developer.spotify.com/documentation/general/guides/app-settings/).


<ins>**Notes:**</ins>
* [**Here**](https://developer.spotify.com/documentation/general/guides/authorization-guide/#client-credentials-flow) you can find the article in which Spotify explains how the `access token` works. In addition, it also mentions different workflows that can be followed in order to get the token. 

In [5]:
access_token = hf.auth()

Client_ID:  6d9cda272a144d6988f08949e9f4cad9
Client_Secret:  28eb8fce5c3448acae7406415e84d1d9


## 4. Getting Data From Spotify
---
[**Reference Link**](https://developer.spotify.com/documentation/web-api/reference/) to make the API calls

### 4.1 `Search` function.
---
There´s a function inside the `helper_functions` library called `search_spotify`. The purpose of this function is to do searches using the Spotify API and get a JSON file in return with the inforamtion that was requested. 
Using this function, we can search information about albums, artists, playlists, tracks, shows and episodes.

This function accepts 3 parameters:
* `access_token`: The json response we get after running the `hf.auth()`´function.
* `query`: The term we want to look for.
* `search_type`: Add the string 'albums', 'artists', 'playlists', 'tracks', 'shows' or 'episodes' depending on what result you're looking to get.

<ins>**Notes:**</ins>
* Click [**here**](https://developer.spotify.com/documentation/web-api/reference/#category-search) to learn more about the "Search" module of the Spotify API.
* If you want to test the "Search" API call in a Spotify Console click [**here**](https://developer.spotify.com/console/get-search-item/).

In [6]:
# Search_Type Options: album , artist, playlist, track, show and episode
albums = hf.search_spotify(access_token, query="Fine Line", search_type='album')
album_cover = albums["albums"]["items"][0]["images"][1]["url"]
album_id = albums["albums"]["items"][0]["id"]
Image(url= f"{album_cover}", width=200, height=200)

### 4.2 `Albums` Function
---
There´s a function inside the `helper_functions` library called `get_tracks`. The purpose of this function is to get all the tracks from an album in a pandas dataframe.

You can select to use one of the next 2 parameters:
* `lookup_id`: If you have the `artist_id` you can put it here.
* `artist`: Name of the artist from which you want to get all their albums. 

<ins>**Notes:**</ins>
* Click [**here**](https://developer.spotify.com/documentation/web-api/reference/#category-albums) to learn more about the "Tracks" module on the Spotify API.
* If you want to test the "Tracks" API call in a Spotify Console click [**here**](https://developer.spotify.com/console/albums/).

In [7]:
all_albums= hf.get_albums(access_token=access_token,lookup_id="66CXWjxzNUsdJxJ2JdwvnR", artist=None)
albums_list = all_albums["album_id"].tolist()
all_albums.head()

AttributeError: module 'helper_functions' has no attribute 'get_albums'

### 4.3 `Tracks` function.
---
There´s a function inside the `helper_functions` library called `get_tracks`. The purpose of this function is to get all the tracks from an album in a pandas dataframe.

This function accepts 2 parameters:
* `access_token`: The json response we get after running the `hf.auth()`´function.
* `album_ids`: The id of a single album. 

<ins>**Notes:**</ins>
* Click [**here**](https://developer.spotify.com/documentation/web-api/reference/#category-tracks) to learn more about the "Tracks" module on the Spotify API.
* If you want to test the "Tracks" API call in a Spotify Console click [**here**](https://developer.spotify.com/console/get-several-tracks/).

In [82]:
songs_from_album = hf.get_tracks(access_token=access_token, album_ids=album_id)
songs_from_album.head()

,track_number,name_of_the_song,duration,explicit,track_id,link,preview,album_id
0,1,Yellow Submarine,159160,False,1tdltVUBkiBCW1C3yB4zyD,https://open.spotify.com/track/1tdltVUBkiBCW1C...,None,0XRZpF083HqgygM0v1hQyE
1,2,Hey Bulldog,191840,False,4epbwW20tHuF9Q6FufAn7Y,https://open.spotify.com/track/4epbwW20tHuF9Q6...,None,0XRZpF083HqgygM0v1hQyE
2,3,Eleanor Rigby,125226,False,6EOKwO6WaLal58MSsi6U4W,https://open.spotify.com/track/6EOKwO6WaLal58M...,None,0XRZpF083HqgygM0v1hQyE
3,4,Love You To,178706,False,69c15XPo8sYUqmnngPtFWj,https://open.spotify.com/track/69c15XPo8sYUqmn...,None,0XRZpF083HqgygM0v1hQyE
4,5,All Together Now,130626,False,4KELrBm2uoyn9tq6fZ5P98,https://open.spotify.com/track/4KELrBm2uoyn9tq...,None,0XRZpF083HqgygM0v1hQyE


### 4.4 Get all the `Tracks` from a single Artist.
---
There´s a method inside the `helper_functions` library called `all_artists_songs()`. The purpose of this method is to get all the tracks from an artist and return them in a pandas dataframe.

This function accepts 2 parameters:
* `at`: The json response we get after running the hf.auth() function.
* `albums_of_artist`: A list wich contains all the albums from a single artist. 

<ins>**Notes:**</ins>
* To get a list with all the albums from a single artist, I recommend to use the `get_albums()` method from the `helper_functions` library. Here's an example:  

```python
all_albums= hf.get_albums(access_token=access_token,lookup_id="66CXWjxzNUsdJxJ2JdwvnR", artist=None)
albums_list = all_albums["album_id"].tolist()
```

In [81]:
albums_list = all_albums["album_id"].tolist()
hf.all_artists_songs(at=access_token, albums_of_artist=albums_list)

,track_number,name_of_the_song,duration,explicit,track_id,link,preview,album_id
0,1,"34+35 Remix (feat. Doja Cat, Megan Thee Stalli...",182437,False,5UqWGhPB6CS0iwSikmtPEJ,https://open.spotify.com/track/5UqWGhPB6CS0iwS...,None,4utF15fvKK0vAaFVGTGXF6
0,1,"34+35 Remix (feat. Doja Cat, Megan Thee Stalli...",182437,True,3jjBPF4NDMCU51psU8JPpR,https://open.spotify.com/track/3jjBPF4NDMCU51p...,None,11X2d5C6rFBFZZUOCJLPt9
0,1,shut up,157710,True,3ko9jEHNt4O5fjKN65ULJp,https://open.spotify.com/track/3ko9jEHNt4O5fjK...,None,3euz4vS7ezKGnNSwgyvKcd
1,2,34+35,173710,True,6Im9k8u9iIzKMrmV7BWtlF,https://open.spotify.com/track/6Im9k8u9iIzKMrm...,None,3euz4vS7ezKGnNSwgyvKcd
2,3,motive (with Doja Cat),167989,True,5GkQIP5mWPi4KZLLXeuFTT,https://open.spotify.com/track/5GkQIP5mWPi4KZL...,None,3euz4vS7ezKGnNSwgyvKcd
...,...,...,...,...,...,...,...,...
7,8,The Way,227026,False,06EL94D0TA27Ik0Ke5usbj,https://open.spotify.com/track/06EL94D0TA27Ik0...,None,5xSvNPstcxHtR4ap2vvN8A
8,9,You’ll Never Know,214280,False,4PqIj0WOfPAq4QAvisjgpd,https://open.spotify.com/track/4PqIj0WOfPAq4QA...,None,5xSvNPstcxHtR4ap2vvN8A
9,10,Almost Is Never Enough,327773,False,63bo7NDoWdMFXufhsYOxwG,https://open.spotify.com/track/63bo7NDoWdMFXuf...,None,5xSvNPstcxHtR4ap2vvN8A
10,11,Popular Song,200213,False,3iugMJEdfE58OpI7WGM38w,https://open.spotify.com/track/3iugMJEdfE58OpI...,None,5xSvNPstcxHtR4ap2vvN8A
